In [3]:
from pathlib import Path
import pandas as pd
import time

In [4]:
#This is the table of all MOUS words, produced by 'master_table.ipynb'
mous_words = pd.read_csv('/home/neel/Desktop/MOUS_hierarchical-representations/mous_words_syllable_bigram_frequencies.csv')
#This is the subject directory where events.tsv and transcription.csv files are stored. 
source = Path('/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source')
mous_words.rename(columns={'Word':'Transcription'}, inplace=True)
mous_words

,Transcription,Syllables,Syllable Occurrence Counts,Bigrams,Bigram Occurrence Counts,Zipf,FREQcount,Lg10WF
0,toen,"[""'tun""]",[47826.0],"['to', 'oe', 'en']","[2236591.0, 837795.0, 8542439.0]",6.032333,47514.0,4.676822
1,de,"[""'d@""]",[1061177.0],['de'],[2559661.0],7.381291,1061177.0,6.025788
2,barkeeper,"[""'bAr"", 'ki', 'p@r']","[3329.0, 3669.0, 20167.0]","['ba', 'ar', 'rk', 'ke', 'ee', 'ep', 'pe', 'er']","[584273.0, 1754491.0, 2174749.0, 3143026.0, 94...",3.622674,184.0,2.264818
3,die,"[""'di""]",[325903.0],"['di', 'ie']","[1050254.0, 2076673.0]",6.853871,315041.0,5.498367
4,irritante,NaN,[],"['ir', 'rr', 'ri', 'it', 'ta', 'an', 'nt', 'te']","[2884214.0, 200692.0, 1787712.0, 333345.0, 892...",3.284921,84.0,1.924279
...,...,...,...,...,...,...,...,...
1937,blije,NaN,[],"['bl', 'li', 'ij', 'je']","[152155.0, 3369152.0, 1639122.0, 913788.0]",3.485836,134.0,2.127105
1938,jongeren,NaN,[],"['jo', 'on', 'ng', 'ge', 'er', 're', 'en']","[866375.0, 2451854.0, 2927082.0, 3692145.0, 67...",3.965097,406.0,2.608526
1939,intens,"['In', ""'tEns""]","[39087.0, 264.0]","['in', 'nt', 'te', 'en', 'ns']","[2264626.0, 1477178.0, 3522714.0, 8542439.0, 6...",3.778748,264.0,2.421604
1940,plezier,"['pl@', ""'zir""]","[5981.0, 5884.0]","['pl', 'le', 'ez', 'zi', 'ie', 'er']","[97171.0, 4397946.0, 100963.0, 123482.0, 20766...",5.093966,5475.0,3.738384


In [5]:
import time
#sort
subjects = [subject for subject in source.iterdir() if subject.name.startswith('sub-A')]
subjects.sort()


In [8]:
# Merge `mous_words` with each transcription to get the Zipf word frequencies for each word. 
for i,subject in enumerate(subjects):
    if subject.name.startswith('sub-A'):
        transcription_path = subject / 'func' / f'{subject.name}_transcription.csv'
        if transcription_path.exists():
            transcription = pd.read_csv(transcription_path)
            #print(transcription.columns)  # Debug: Print columns to check if 'Transcription' exists
            if 'Transcription' in transcription.columns:
                transcription['Zipf'] = transcription['Transcription'].map(mous_words.set_index('Transcription')['Zipf'])
                transcription['Lg10WF'] = transcription['Transcription'].map(mous_words.set_index('Transcription')['Lg10WF'])
                # Only keep AlignOnset, Transcription, and Zipf columns
                transcription = transcription[['AlignOnset', 'Transcription', 'Zipf', 'Lg10WF']]
                transcription.to_csv(subject / 'func' / f'{subject.name}_word_frequencies.csv', index=False)
            else:
                print(f"'Transcription' column not found in {transcription_path}")
